In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
import glob
import cv2
import os
import matplotlib.pyplot as plt

In [0]:
# set data path
base_dir = "drive/My Drive/Colab Notebooks/"
bottleneck_dir = os.path.join(base_dir, 'bottleneck')
xcept_bottleneck_dir = os.path.join(bottleneck_dir, 'xcept')
incept_bottleneck_dir = os.path.join(bottleneck_dir, 'incept')

In [0]:
train_df = pd.read_csv(base_dir + 'train.csv')

train_labels = train_df['label'].values


In [0]:
train_labels[:50]

array([1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 1])

从网络下载training图片。下面是真正的训练

In [0]:
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D,Dropout,Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

Using TensorFlow backend.


In [0]:

def get_callbacks(name_weights, patience_lr):
    path = name_weights
    mcp_save = ModelCheckpoint(name_weights,save_best_only=True, 
                               monitor='val_loss', mode='min')
    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, 
                                  patience=patience_lr, verbose=1,
                                 min_delta=1e-4, mode='min')
    return [mcp_save,reduce_lr]
    

In [0]:
def save_result(name,predictions,ids=[]):
  result = pd.DataFrame({'label': np.squeeze(predictions)})
  if len(ids)==0:
    result['id'] = pd.Series(range(1,len(predictions)+1),index=range(len(predictions)))
  else:
    result['id'] = pd.Series(ids,index=range(len(predictions)))
  result.to_csv(name,index=False, header=True)

In [0]:
train_data = np.load(open(xcept_bottleneck_dir+'/train.npy','rb'))
test_data = np.load(open(xcept_bottleneck_dir+'/test.npy','rb'))


print(len(train_data),len(test_data))

25000 12500


In [0]:
print(train_data.shape)

(25000, 2048)


In [0]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(train_data,train_labels,test_size=0.2,random_state=42)

In [0]:
model = Sequential()
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])


#train
BATCH_SIZE = 100
weight_path = base_dir + 'weights/2/' #save to drive

train_pred_1 = np.zeros(train_df.shape[0])
test_pred_1 = np.zeros(len(test_data))
                       
            
name_weights_1 = os.path.join(weight_path,'best_model_1_weights.h5')
callbacks = get_callbacks(name_weights_1,patience_lr=5)                  

history = model.fit(train_X,train_y,
                    epochs=100,
                    batch_size=BATCH_SIZE,
                    validation_data=(val_X, val_y),
                    callbacks=callbacks)

Train on 20000 samples, validate on 5000 samples
Epoch 1/100
20000/20000 [==============================] - 1s 52us/step - loss: 0.1272 - acc: 0.9751 - val_loss: 0.0422 - val_acc: 0.9916
Epoch 2/100
20000/20000 [==============================] - 1s 42us/step - loss: 0.0377 - acc: 0.9918 - val_loss: 0.0295 - val_acc: 0.9920
Epoch 3/100
20000/20000 [==============================] - 1s 42us/step - loss: 0.0293 - acc: 0.9928 - val_loss: 0.0258 - val_acc: 0.9924
Epoch 4/100
20000/20000 [==============================] - 1s 42us/step - loss: 0.0255 - acc: 0.9929 - val_loss: 0.0234 - val_acc: 0.9928
Epoch 5/100
20000/20000 [==============================] - 1s 43us/step - loss: 0.0229 - acc: 0.9936 - val_loss: 0.0227 - val_acc: 0.9924
Epoch 6/100
20000/20000 [==============================] - 1s 43us/step - loss: 0.0212 - acc: 0.9940 - val_loss: 0.0218 - val_acc: 0.9928
Epoch 7/100
20000/20000 [==============================] - 1s 42us/step - loss: 0.0209 - acc: 0.9934 - val_loss: 0.0214 - v

In [0]:
model = Sequential()
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
adam = optimizers.Adam(lr=0.0005, decay=1e-6)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [0]:
#train
BATCH_SIZE = 100
weight_path = base_dir + 'weights/2/' #save to drive

train_pred_1 = np.zeros(train_df.shape[0])
test_pred_1 = np.zeros(len(test_data))
                       
            
name_weights_1 = os.path.join(weight_path,'best_model_1_weights.h5')
callbacks = get_callbacks(name_weights_1,patience_lr=5)                  

history = model.fit(train_X,train_y,
                    epochs=50,
                    batch_size=BATCH_SIZE,
                    validation_data=(val_X, val_y),
                    callbacks=callbacks)

model.load_weights(name_weights_1)  
train_pred_1 = model.predict(train_data)
test_pred_1 = model.predict(test_data)
save_result(os.path.join(weight_path,'xcept_test.csv'),test_pred_1,'')
save_result(os.path.join(weight_path,'xcept_train.csv'),train_pred_1,train_data)

Train on 20000 samples, validate on 5000 samples
Epoch 1/50
20000/20000 [==============================] - 1s 59us/step - loss: 0.3014 - acc: 0.9303 - val_loss: 0.1235 - val_acc: 0.9882
Epoch 2/50
20000/20000 [==============================] - 1s 28us/step - loss: 0.0953 - acc: 0.9884 - val_loss: 0.0685 - val_acc: 0.9914
Epoch 3/50
20000/20000 [==============================] - 1s 28us/step - loss: 0.0631 - acc: 0.9900 - val_loss: 0.0503 - val_acc: 0.9918
Epoch 4/50
20000/20000 [==============================] - 1s 27us/step - loss: 0.0492 - acc: 0.9906 - val_loss: 0.0410 - val_acc: 0.9918
Epoch 5/50
20000/20000 [==============================] - 1s 28us/step - loss: 0.0416 - acc: 0.9917 - val_loss: 0.0358 - val_acc: 0.9918
Epoch 6/50
20000/20000 [==============================] - 1s 27us/step - loss: 0.0372 - acc: 0.9914 - val_loss: 0.0323 - val_acc: 0.9922
Epoch 7/50
20000/20000 [==============================] - 1s 28us/step - loss: 0.0336 - acc: 0.9919 - val_loss: 0.0301 - val_acc:

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
print(train_pred_1.shape,train_labels.shape)
save_result(os.path.join(weight_path,'xcept_test.csv'),test_pred_1)
save_result(os.path.join(weight_path,'xcept_train.csv'),train_pred_1,train_labels)

(25000, 1) (25000,)


In [0]:
train_data = np.load(open(incept_bottleneck_dir+'/train.npy','rb'))
test_data = np.load(open(incept_bottleneck_dir+'/test.npy','rb'))

In [0]:
train_X, val_X, train_y, val_y = train_test_split(train_data,train_labels,test_size=0.2,random_state=43)
model = Sequential()
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
adam = optimizers.Adam(lr=0.0005, decay=1e-6)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [0]:
#train
train_pred_2 = np.zeros(train_df.shape[0])
test_pred_2 = np.zeros(len(test_data))
                       
            
name_weights_2 = os.path.join(weight_path,'best_model_2_weights.h5')
callbacks = get_callbacks(name_weights_2,patience_lr=5)                  

history = model.fit(train_X,train_y,
                    epochs=50,
                    batch_size=BATCH_SIZE,
                    validation_data=(val_X, val_y),
                    callbacks=callbacks)

model.load_weights(name_weights_2)    
train_pred_2 = model.predict(train_data)
test_pred_2 = model.predict(test_data)
save_result(os.path.join(weight_path,'incept_test.csv'),test_pred_2)
save_result(os.path.join(weight_path,'incept_train.csv'),train_pred_2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/50
20000/20000 [==============================] - 1s 62us/step - loss: 0.2704 - acc: 0.9073 - val_loss: 0.0855 - val_acc: 0.9880
Epoch 2/50
20000/20000 [==============================] - 1s 26us/step - loss: 0.0788 - acc: 0.9862 - val_loss: 0.0506 - val_acc: 0.9916
Epoch 3/50
20000/20000 [==============================] - 1s 27us/step - loss: 0.0546 - acc: 0.9888 - val_loss: 0.0385 - val_acc: 0.9926
Epoch 4/50
20000/20000 [==============================] - 1s 28us/step - loss: 0.0448 - acc: 0.9892 - val_loss: 0.0324 - val_acc: 0.9930
Epoch 5/50
20000/20000 [==============================] - 1s 27us/step - loss: 0.0402 - acc: 0.9888 - val_loss: 0.0287 - val_acc: 0.9930
Epoch 6/50
20000/20000 [==============================] - 1s 28us/step - loss: 0.0345 - acc: 0.9913 - val_loss: 0.0265 - val_acc: 0.9930
Epoch 7/50
20000/20000 [==============================] - 1s 27us/step - loss: 0.0333 - acc: 0.9908 - val_loss: 0.0248 - val_acc:

In [0]:
print(test_pred_1[:50])
print(test_pred_2[:50])

[[9.9967897e-01]
 [9.9992663e-01]
 [9.9990332e-01]
 [9.9984229e-01]
 [5.7816505e-06]
 [2.3663044e-05]
 [6.0796738e-06]
 [1.6212463e-05]
 [5.0663948e-07]
 [7.3611736e-06]
 [1.6510487e-05]
 [9.9426293e-01]
 [3.4272671e-05]
 [2.2500753e-05]
 [2.1815300e-05]
 [2.9176474e-05]
 [9.8468924e-01]
 [9.9982381e-01]
 [1.3414025e-04]
 [5.5286288e-04]
 [9.9954009e-01]
 [3.8313568e-03]
 [9.9908626e-01]
 [9.9888957e-01]
 [3.5762787e-06]
 [9.9897945e-01]
 [9.9945062e-01]
 [3.1888485e-06]
 [1.1670858e-02]
 [9.9576080e-01]
 [9.9980319e-01]
 [8.9118946e-01]
 [9.9990511e-01]
 [1.7583370e-06]
 [5.8412552e-06]
 [1.2409687e-04]
 [1.4662743e-05]
 [3.8743019e-06]
 [9.5781511e-01]
 [7.5399876e-06]
 [9.9637812e-01]
 [9.9936843e-01]
 [9.9939626e-01]
 [9.9852705e-01]
 [2.8848648e-05]
 [9.9994135e-01]
 [1.2218952e-05]
 [9.9973708e-01]
 [9.9847591e-01]
 [4.5299530e-06]]
[[9.97825384e-01]
 [9.99914169e-01]
 [9.99450922e-01]
 [9.99280810e-01]
 [5.96046448e-07]
 [1.22487545e-05]
 [4.55975533e-06]
 [2.37524509e-05]
 [1.6

In [0]:
# lgb for stacking
import lightgbm as lgb
from sklearn.model_selection import train_test_split
param = {'learning_rate': 0.02,
         'max_depth': 7,
         'n_estimators':500,
         'num_leaves': 15,
         'min_child_samples': 2,
         'min_child_weight':0.01,
         "feature_fraction": 1,
         "bagging_freq": 1,
         "bagging_fraction": 0.4 ,
         'reg_alpha': 0, 
         'reg_lambda': 1.1,
         "boosting": "gbdt",
         "bagging_seed": 11,
         'objective':'binary', # for multi softmax 
         "metric": 'binary_logloss', #multi_logloss for multi classification  
         "verbosity": -1}

train_stack = np.hstack([train_pred_1,train_pred_2])
test_stack = np.hstack([test_pred_1, test_pred_2])

predictions = np.zeros(len(test_data))
X_train, X_val, y_train, y_val = train_test_split(train_stack,train_df['label'].values, test_size=0.2, random_state=44)

trn_data = lgb.Dataset(X_train, y_train)
val_data = lgb.Dataset(X_val, y_val)

num_round = 400
clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=200, early_stopping_rounds = 100) 
predictions = clf.predict(test_stack, num_iteration=clf.best_iteration)
save_result(os.path.join(weight_path,'result.csv'),predictions)
train_predictions = clf.predict(train_stack, num_iteration=clf.best_iteration)
save_result(os.path.join(weight_path,'train_pred.csv'),train_predictions)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[200]	training's binary_logloss: 0.0179653	valid_1's binary_logloss: 0.0203525
[400]	training's binary_logloss: 0.00862777	valid_1's binary_logloss: 0.0122862
Did not meet early stopping. Best iteration is:
[500]	training's binary_logloss: 0.0081491	valid_1's binary_logloss: 0.0123015


In [0]:
predictions = predictions.clip(min=0.005, max=0.995)
save_result(os.path.join(weight_path,'tune.csv'),predictions)